In [1]:
%matplotlib inline
import warnings
import re as re
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import random, sys, os
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

from pandas_profiling import ProfileReport

from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import (train_test_split, RandomizedSearchCV, 
                                     GridSearchCV, KFold, cross_val_predict, 
                                     permutation_test_score, StratifiedKFold) 

from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier,
                              GradientBoostingClassifier, ExtraTreesClassifier)

from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier

Using TensorFlow backend.


In [2]:
# Loading Pickled DataFrames
X_train = pd.read_pickle("Processed_Data/X_train.pkl")
X_test = pd.read_pickle("Processed_Data/X_test.pkl")
y_train = pd.read_pickle("Processed_Data/y_train.pkl")
y_test = pd.read_pickle("Processed_Data/y_test.pkl")
test = pd.read_pickle("Processed_Data/test.pkl")
X_train.columns

Index(['Region', 'Gender', 'Married', 'Children', 'Occupation',
       'Mode_transport', 'cases/1M', 'Deaths/1M', 'comorbidity', 'Age',
       'Coma score', 'Pulmonary score', 'cardiological pressure', 'Diuresis',
       'Platelets', 'HBB', 'd-dimer', 'Heart rate', 'HDL cholesterol',
       'Charlson Index', 'Blood Glucose', 'Insurance', 'salary', 'FT/month'],
      dtype='object')

In [3]:
# Converting to binary values and use binary_crossentropy
# y_bin_train = y_train > 0.5 
# y_bin_train = y_bin_train + 0

# y_bin_test = y_test > 0.5
# y_bin_test = y_bin_test + 0

# If using direct probabilities
y_bin_train = y_train
y_bin_test = y_test

In [4]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
test = scaler.transform(test)

In [5]:
# cols = ['Region', 'Married', 'Children', 'Deaths/1M', 'Insurance', 'salary', 'Occupation', 'Gender']
# X_train.drop(columns=cols, inplace=True)
# X_test.drop(columns=cols, inplace=True)
# test.drop(columns=cols, inplace=True)
# X_train.columns

In [6]:
def baseline_model():
    model = Sequential()
    #get number of columns in training data
    n_cols = X_train.shape[1]
    #add model layers
    model.add(Dense(128, activation='relu', input_shape=(n_cols,)))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
model = baseline_model()
model.fit(X_train, y_bin_train, epochs=50, batch_size=32)
score = model.evaluate(X_test, y_bin_test)
model.save('Models/Task-1-reg.h5')
predictions = model.predict(test)
print(score)


Epoch 1/50
8571/8571 [==============================] - 1s 152us/step - loss: 0.0113
Epoch 2/50
8571/8571 [==============================] - 1s 111us/step - loss: 0.0096
Epoch 3/50
8571/8571 [==============================] - 1s 110us/step - loss: 0.0093
Epoch 4/50
8571/8571 [==============================] - 1s 110us/step - loss: 0.0089
Epoch 5/50
8571/8571 [==============================] - 1s 110us/step - loss: 0.0089
Epoch 6/50
8571/8571 [==============================] - 1s 114us/step - loss: 0.0088
Epoch 7/50
8571/8571 [==============================] - 1s 112us/step - loss: 0.0086
Epoch 8/50
8571/8571 [==============================] - 1s 114us/step - loss: 0.0087
Epoch 9/50
8571/8571 [==============================] - 1s 112us/step - loss: 0.0086
Epoch 10/50
8571/8571 [==============================] - 1s 110us/step - loss: 0.0085
Epoch 11/50
8571/8571 [==============================] - 1s 91us/step - loss: 0.0084
Epoch 12/50
8571/8571 [==============================] - 1s 103

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               3200      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)               

In [9]:
predictions = model.predict(test)
predictions = np.squeeze(predictions, axis=(1,))
predictions

array([0.52716863, 0.53764963, 0.50077134, ..., 0.495145  , 0.52048343,
       0.49574503], dtype=float32)

In [10]:
test_people = pd.read_excel('Test_dataset.xlsx')
final_df = pd.DataFrame({'people_ID':test_people['people_ID'], 'Infect_Prob': predictions*100})
final_df.head()

,people_ID,Infect_Prob
0,5942,52.716862
1,18664,53.764961
2,5603,50.077133
3,5649,51.810932
4,5099,50.798637


In [11]:
final_df.to_csv('Results/Task-1_Results.csv', index=False)